# To Summarize
1) Since our data was small in size and did not have user questions. I used a pre-trained QA Model that uses the course names and course description to generate a hypothetical question

2) I then transformed these questions into a numerical vector (encoding) to be able to compare any user asked question to the most similar generated question that we have (Note: this is done since we cannot compare text to text, we need to change them to numbers first)

3) Finally I developed an FAISS index (like a search engine) so I can output to the user all the information about the course efficiently

In [ ]:
# Mount Google Drive to get data files
from google.colab import drive 
drive.mount('/content/gdrive')

# **Data Preparation**

In [2]:
import json
import glob
import pandas as pd
import numpy as np

1. read df, try do to question model
2. use text column in df, do recommendation model

In [3]:
df = pd.read_csv('/Users/briantsai/Downloads/XNproject/Gigturbodataset.csv',encoding='latin1')


In [4]:
df

,learning_platform,career_path,skill_level,course_name,course_description,course_detail,course_instructors,course_syllabus,prerequisites,related_courses,course_reviews,faq
0,Udacity,Data Science,Beginner,Intro to Data Science | Udacity Free Courses,The Introduction to Data Science class will su...,"{""course_detail"": [""Course Cost: Free"", ""Timel...","{""course_instructors"": [""Dave Holtz, Instructo...","{""course_syllabus"": [""lesson 1 Introduction to...","{""prerequisites"": [""Strong interest in data sc...","{""related_courses"": [""Introduction to Python P...","{""course_reviews"": []}","{""faq"": []}"
1,Udacity,Data Science,Beginner,Learn SQL | SQL Online Course,"Master SQL, the core language for Big Data ana...","{""course_detail"": [""Estimated Time: 2 months, ...","{""course_instructors"": [""Malavica Sridhar, Pro...","{""course_syllabus"": [""Perform analysis on data...","{""prerequisites"": [""Basic understanding of dat...","{""related_courses"": [""Business Analytics - Nan...","{""course_reviews"": [""FJ S. - 5.0 stars - The o...","{""faq"": []}"
2,Udacity,Data Science,Beginner,Business Analytics Online Course | Learn Busin...,Gain foundational data skills applicable to an...,"{""course_detail"": [""Estimated Time: 3 Months, ...","{""course_instructors"": [""Josh Bernhard, Data S...","{""course_syllabus"": [""In this program, you wil...","{""prerequisites"": [""This is an introductory pr...","{""related_courses"": [""Marketing Analytics - Na...","{""course_reviews"": [""Mashael A. - 5.0 stars - ...","{""faq"": []}"
3,Udacity,Data Science,Beginner,Learn Python for Data Science,Prepare for a data science career by learning ...,"{""course_detail"": [""Estimated Time: 3 Months, ...","{""course_instructors"": [""Josh Bernhard, Data S...","{""course_syllabus"": [""Learn the programming fu...","{""prerequisites"": [""There are no prerequisites...","{""related_courses"": [""Data Analyst - Nanodegre...","{""course_reviews"": [""David M A. - 5.0 stars - ...","{""faq"": []}"
4,Coursera,Data Science,Beginner,Python for Everybody | Coursera,This Specialization builds on the success of t...,"{""course_detail"": ['Shareable Certificate, Ear...","{""course_instructors"": ['Charles Russell Sever...","{""course_syllabus"": ['Install Python and write...","{""prerequisites"": []}","{""related_courses"": ['Programming for Everybod...","{""course_reviews"": ['Coursera helped me hone m...","{""faq"": [""What is the refund policy?If you sub..."
5,Coursera,Data Science,Beginner,Data Science Math Skills | Coursera,Data science courses contain math - no avoidin...,"{""course_detail"": ['Flexible deadlines. Reset ...","{""course_instructors"": ['Daniel EggerExecutive...","{""course_syllabus"": [""Hours to complete18 minu...","{""prerequisites"": []}","{""related_courses"": []}","{""course_reviews"": [""I got hooked with everyth...","{""faq"": ['When will I have access to the lectu..."
6,Coursera,Data Science,Beginner,Inferential Statistics | Coursera,This course covers commonly used statistical i...,"{""course_detail"": ['Flexible deadlinesFlexible...","{""course_instructors"": ['Mine Ãâ¡etinkaya-Ru...","{""course_syllabus"": ['Hours to complete20 minu...","{""prerequisites"": []}","{""related_courses"": ['Introduction to Probabil...","{""course_reviews"": ['Coursera gives me the fle...","{""faq"": [""When will I have access to the lectu..."
7,Coursera,Data Science,Beginner,IBM Data Analyst Professional Certificate | Co...,Gain the job-ready skills for an entry-level d...,"{""course_detail"": ['Shareable Certificate Earn...","{""course_instructors"": ['Rav AhujaGlobal Progr...","{""course_syllabus"": ['Demonstrate proficiency ...","{""prerequisites"": []}","{""related_courses"": ['Introduction to Data Ana...","{""course_reviews"": ['Courses on Coursera playe...","{""faq"": ['What is the refund policy?If you sub..."
8,Udacity,Data Science,Intermediate,Data Visualization Course | Data Science,"Combine data, visuals, and narrative to tell 

# Recommendation Model

#### Step 2:  Creating a combined description of every student in the dataset.

In [31]:
df['course_name'] = df['course_name'].str.rsplit('|', n=1).str.get(0)

In [33]:
descriptions =df['learning_platform'] +' '+ df['career_path'] + ' ' + df['skill_level'] +' '+ df['course_name'] +' '+ df['course_instructors'] + ' ' + df['course_description'] +' '+ df['course_detail'] + ' ' + df['prerequisites'] +' '+ df['related_courses'] + ' ' + df['course_reviews'].apply(str)
#Printing the first description
descriptions[0]

'Udacity Data Science Beginner Intro to Data Science  {"course_instructors": ["Dave Holtz, Instructor", "Cheng-Han Lee, Instructor"]} The Introduction to Data Science class will survey the foundational topics in data science, namely: Data Manipulation, Data Analysis with Statistics and Machine Learning, Data Communication with Information Visualization, Data at Scale -- Working with Big Data, The class will focus on breadth and present the topics briefly instead of focusing on a single topic in depth. This will give you the opportunity to sample and apply the basic techniques of data science., This course is also a part of our Data Analyst Nanodegree. {"course_detail": ["Course Cost: Free", "Timeline: Approx. 2 months", "Skill Level: intermediate"]} {"prerequisites": ["Strong interest in data science", "Background in intro level statistics", "Python programming experience", "Or understanding of programming concepts such as variables, functions, loops, and basic python data structures l

#### Step 3: Creating a similarity matrix between the students.

In [34]:
# import TfidfVector from sklearn.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [36]:
def create_similarity_matrix(new_description, overall_descriptions):
#Append the new description to the overall set.
        overall_descriptions.append(new_description)
        # Define a tfidf vectorizer and remove all stopwords.
        tfidf = TfidfVectorizer(stop_words="english")
        #Convert tfidf matrix by fitting and transforming the data.
        tfidf_matrix = tfidf.fit_transform(overall_descriptions)
        # output the shape of the matrix.
        tfidf_matrix.shape
        # calculating the cosine similarity matrix.
        cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

        return cosine_sim

#### Step 4: Define get_recommendations() function.

In [42]:
def get_recommendations(new_description,overall_descriptions):
    # create the similarity matrix
    cosine_sim = create_similarity_matrix(new_description,overall_descriptions)
    # Get pairwise similarity scores of all the students with new student.
    sim_scores = list(enumerate(cosine_sim[-1]))
    # Sort the descriptions based on similarity score.
    sim_scores = sorted(sim_scores,key =lambda x:x[1],reverse= True )
    # Get the scores of top 10 descriptions.
    sim_scores = sim_scores[1:10]
    # Get the student indices.
    indices = [i[0]for i in sim_scores]

    return df.iloc[indices]

#### Step 5: Recommend the course to a new student.

In [45]:
new_description = pd.Series('Udacity Data Science Beginner')
create_similarity_matrix(new_description, descriptions)


/var/folders/7h/l_zr79r1053dk59rg6fl_cnc0000gn/T/ipykernel_42904/3694499792.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  overall_descriptions.append(new_description)


array([[1.        , 0.14105855, 0.24839373, ..., 0.10266749, 0.1288965 ,
        0.11321358],
       [0.14105855, 1.        , 0.37540309, ..., 0.12755075, 0.15447604,
        0.22656707],
       [0.24839373, 0.37540309, 1.        , ..., 0.22672403, 0.24828419,
        0.22850271],
       ...,
       [0.10266749, 0.12755075, 0.22672403, ..., 1.        , 0.47016443,
        0.21860556],
       [0.1288965 , 0.15447604, 0.24828419, ..., 0.47016443, 1.        ,
        0.2495125 ],
       [0.11321358, 0.22656707, 0.22850271, ..., 0.21860556, 0.2495125 ,
        1.        ]])

In [46]:
get_recommendations(new_description,descriptions)

/var/folders/7h/l_zr79r1053dk59rg6fl_cnc0000gn/T/ipykernel_42904/3694499792.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  overall_descriptions.append(new_description)


,learning_platform,career_path,skill_level,course_name,course_description,course_detail,course_instructors,course_syllabus,prerequisites,related_courses,course_reviews,faq
9,Udacity,Data Science,Intermediate,Learn R Programming,Prepare for a data science career by learning ...,"{""course_detail"": [""Estimated Time: 3 Months, ...","{""course_instructors"": [""Josh Bernhard, Data S...","{""course_syllabus"": [""Learn the programming fu...","{""prerequisites"": [""There are no prerequisites...","{""related_courses"": [""Programming for Data Sci...","{""course_reviews"": [""Mawejje H. - 5.0 stars - ...","{""faq"": []}"
3,Udacity,Data Science,Beginner,Learn Python for Data Science,Prepare for a data science career by learning ...,"{""course_detail"": [""Estimated Time: 3 Months, ...","{""course_instructors"": [""Josh Bernhard, Data S...","{""course_syllabus"": [""Learn the programming fu...","{""prerequisites"": [""There are no prerequisites...","{""related_courses"": [""Data Analyst - Nanodegre...","{""course_reviews"": [""David M A. - 5.0 stars - ...","{""faq"": []}"
33,Udacity,Software Engineering,Intermediate,Become a Full Stack Java Developer,Meet the growing demand for full stack develop...,"{""course_detail"": [""Estimated Time: 4 Months, ...","{""course_instructors"": [""Rachel Manning, FULL ...","{""course_syllabus"": [""Full Stack JavaScript De...","{""prerequisites"": [""HTML, CSS, Basic JavaScrip...","{""related_courses"": [""Front End Web Developer ...","{""course_reviews"": []}","{""faq"": []}"
10,Udacity,Data Science,Intermediate,How to Become a Data Scientist,Gain real-world data science experience with p...,"{""course_detail"": [""Estimated Time: 4 Months, ...","{""course_instructors"": [""Josh Bernhard, Data S...","{""course_syllabus"": [""You will master the skil...","{""prerequisites"": [""We recommend students are ...","{""related_courses"": [""Computer Vision - Nanode...","{""course_reviews"": [""Zhaoyun M. - 5.0 stars - ...","{""faq"": []}"
36,Udacity,Software Engineering,Expert,Deep Learning,Deep learning is driving advances in artificia...,"{""course_detail"": [""Estimated Time: 4 months, ...","{""course_instructors"": [""Mat Leonard, Instruct...","{""course_syllabus"": [""Deep Learning"", ""Become ...","{""prerequisites"": [""This program has been crea...","{""related_courses"": [""Computer Vision - Nanode...","{""course_reviews"": [""Vyom S. - 5.0 stars - I'v...","{""faq"": []}"
11,Udacity,Data Science,Intermediate,Data Engineering I How to Become a Data Engineer,Data Engineering is the foundation for the new...,"{""course_detail"": [""Estimated Time: 5 Months, ...","{""course_instructors"": [""Amanda Moran, Develop...","{""course_syllabus"": [""Learn to design data mod...","{""prerequisites"": [""To be successful in this p...","{""related_courses"": [""AWS Cloud Architect - Na...","{""course_reviews"": [""Justin W. - 5.0 stars - O...","{""faq"": []}"
32,Udacity,Software Engineering,Beginner,Learn C++ Online,"Learn C++, a high-performance programming lang...","{""course_detail"": [""Estimated Time: 4 Months, ...","{""course_instructors"": [""David Silver, Head of...","{""course_syllabus"": [""Learn C++ by Coding Five...","{""prerequisites"": [""To optimize your chances o...","{""related_courses"": [""Sensor Fusion Engineer -...","{""course_reviews"": [""Chloe H. - 5.0 stars - Th...","{""faq"": []}"
2,Udacity,Data Science,Beginner,Business Analytics Online Course,Gain foundational data skills applicable to an...,"{""course_detail"": [""Estimated Time: 3 Months, ...","{""course_instructors"": [""Josh Bernhard, Data S...","{""course_syllabus"": [""In this program, you wil...","{""prerequisites"": [""This is an introductory pr...","{""related_courses"": [""Marketing Analytics - Na...","{""course_reviews"": [""Mashael A. - 5.0 stars - ...","{""faq"": []}"
1,Udacity,Data Science,Beginner,Learn SQL,"Master SQL, the core language for Big Data ana...","{""course_detail"": [""Estimated Time: 2 months, ...","{""cou

# Question Generation

In [ ]:
# a model refered by sponsor
https://huggingface.co/deepset/roberta-base-squad2

In [ ]:
# Fetching and Preparing the data

path = r'/content/gdrive/My Drive/Colab Notebooks/Work/QA Model' # path to data files - you can use your file path / data

# function to get all the datasets, clean and prepare them
def prep_data(path):
  all_files = glob.glob(path + "/*.csv") # Getting all the datasets
  global li # saving the list of dataframes as a global variable
  li = [] # empty list
  for filename in all_files: # loop to read each dataframe and save it in the list
    df_prep = pd.read_csv(filename, index_col=None, header=0)
    li.append(df_prep)

prep_data(path) # apply the function

df = pd.concat(li) # concat the list of dataframes
df.reset_index(drop=True, inplace=True)

# --- Note you can use and build on the below code for more cleaning but I did not need it for this application

# splitting and cleaning the course info, if you want to use for organizing the datasets
#df2 = df[['course_detail','course_instructors','course_syllabus', 'prerequisites', 'related_courses', 'course_reviews', 'faq']]
#splits = [df2[col].str.split(':', 1, expand=True).add_prefix(col) for col in df2.columns]

# --- 

In [5]:
# Extracting the elements I will use for the models

df2 = df.iloc[:, [0,1,2,3,4]] # I use those columns in our models
df2['course_name'] = df2['course_name'].str.rsplit('|', n=1).str.get(0) # removing the extra section in course_name
df2.head()

/var/folders/7h/l_zr79r1053dk59rg6fl_cnc0000gn/T/ipykernel_42904/1579087156.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['course_name'] = df2['course_name'].str.rsplit('|', n=1).str.get(0) # removing the extra section in course_name


,learning_platform,career_path,skill_level,course_name,course_description
0,Udacity,Data Science,Beginner,Intro to Data Science,The Introduction to Data Science class will su...
1,Udacity,Data Science,Beginner,Learn SQL,"Master SQL, the core language for Big Data ana..."
2,Udacity,Data Science,Beginner,Business Analytics Online Course,Gain foundational data skills applicable to an...
3,Udacity,Data Science,Beginner,Learn Python for Data Science,Prepare for a data science career by learning ...
4,Coursera,Data Science,Beginner,Python for Everybody,This Specialization builds on the success of t...


# **Model 1 Training - Question Generation**

Here I use two pretrained models to generate a question that the user could potentially ask (note: you can use other pre-trained models or re-train a model but since the data we have is small I will use this approach)


Please note that there are many strategies you could take in terms of model building. In this notebook I only experiment with some ideas


# Model 1a - Pretrained model 1a
Link: https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap



In [7]:
%%capture 
# capture is to hide output when loading

!pip install transformers
!pip install transformers[sentencepiece]
!pip install sentencepiece
!pip install torch

In [6]:
%%capture

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer

In [7]:
# tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

# If you have and want to use GPU
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#assert device == torch.device('cuda'), "Not using CUDA. Set: Runtime > Change runtime type > Hardware Accelerator: GPU"

/Users/briantsai/opt/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [8]:
# Model 1a

# function for question generation given a context (course description) and answer (course name) 
def get_question(answer, context, max_length=64): # max_length could be changed
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

context = df2['course_description'][1] # from our data I am using course_description as context
answer = df2['course_name'][1] # from our data I am using course_name as answer

get_question(answer, context) 

'<pad> question: What is the core language for Big Data analysis?</s>'

# Model 1b - Pretrained model 1b
Link: https://huggingface.co/iarfmoose/t5-base-question-generator


In [9]:
%%capture

!git clone https://github.com/amontgomerie/question_generator/
!cd question_generator

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
pip install spacy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 6.5 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 10.5 MB/s eta 0:00:01
  Using cached cymem-2.0.6-cp39-cp39-macosx_10_9_x86_64.whl (32 kB)
     |████████████████████████████████| 2.7 MB 62.4 MB/s eta 0:00:01
     |████████████████████████████████| 457 kB 124.9 MB/s eta 0:00:01
     |████████████████████████████████| 645 kB 147.1 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 104.8 MB/s eta 0:00:01
  Using cached wasabi-0.9.1-py3-none-any.whl (26 kB)
  Using cached preshed-3.0.6-cp39-cp39-macosx_10_9_x86_64.whl (106 kB)
     |████████████████████████████████| 5.8 MB 146.5 MB/s eta 0:00:01

In [21]:
!python3 -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 12.8 MB 3.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [22]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [23]:
# %load questiongenerator.py
import en_core_web_sm
import json
import numpy as np
import random
import re
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
)
from typing import Any, List, Mapping, Tuple


In [24]:
class QuestionGenerator:
    """A transformer-based NLP system for generating reading comprehension-style questions from
    texts. It can generate full sentence questions, multiple choice questions, or a mix of the
    two styles.

    To filter out low quality questions, questions are assigned a score and ranked once they have
    been generated. Only the top k questions will be returned. This behaviour can be turned off
    by setting use_evaluator=False.
    """

    def __init__(self) -> None:

        QG_PRETRAINED = "iarfmoose/t5-base-question-generator"
        self.ANSWER_TOKEN = "<answer>"
        self.CONTEXT_TOKEN = "<context>"
        self.SEQ_LENGTH = 512

        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

        self.qg_tokenizer = AutoTokenizer.from_pretrained(
            QG_PRETRAINED, use_fast=False)
        self.qg_model = AutoModelForSeq2SeqLM.from_pretrained(QG_PRETRAINED)
        self.qg_model.to(self.device)
        self.qg_model.eval()

        self.qa_evaluator = QAEvaluator()

    def generate(
        self,
        article: str,
        use_evaluator: bool = True,
        num_questions: bool = None,
        answer_style: str = "all"
    ) -> List:
        """Takes an article and generates a set of question and answer pairs. If use_evaluator
        is True then QA pairs will be ranked and filtered based on their quality. answer_style
        should selected from ["all", "sentences", "multiple_choice"].
        """

        print("Generating questions...\n")

        qg_inputs, qg_answers = self.generate_qg_inputs(article, answer_style)
        generated_questions = self.generate_questions_from_inputs(qg_inputs)

        message = "{} questions doesn't match {} answers".format(
            len(generated_questions), len(qg_answers)
        )
        assert len(generated_questions) == len(qg_answers), message

        if use_evaluator:
            print("Evaluating QA pairs...\n")
            encoded_qa_pairs = self.qa_evaluator.encode_qa_pairs(
                generated_questions, qg_answers
            )
            scores = self.qa_evaluator.get_scores(encoded_qa_pairs)

            if num_questions:
                qa_list = self._get_ranked_qa_pairs(
                    generated_questions, qg_answers, scores, num_questions
                )
            else:
                qa_list = self._get_ranked_qa_pairs(
                    generated_questions, qg_answers, scores
                )

        else:
            print("Skipping evaluation step.\n")
            qa_list = self._get_all_qa_pairs(generated_questions, qg_answers)

        return qa_list

    def generate_qg_inputs(self, text: str, answer_style: str) -> Tuple[List[str], List[str]]:
        """Given a text, returns a list of model inputs and a list of corresponding answers.
        Model inputs take the form "answer_token <answer text> context_token <context text>" where
        the answer is a string extracted from the text, and the context is the wider text surrounding
        the context.
        """

        VALID_ANSWER_STYLES = ["all", "sentences", "multiple_choice"]

        if answer_style not in VALID_ANSWER_STYLES:
            raise ValueError(
                "Invalid answer style {}. Please choose from {}".format(
                    answer_style, VALID_ANSWER_STYLES
                )
            )

        inputs = []
        answers = []

        if answer_style == "sentences" or answer_style == "all":
            segments = self._split_into_segments(text)

            for segment in segments:
                sentences = self._split_text(segment)
                prepped_inputs, prepped_answers = self._prepare_qg_inputs(
                    sentences, segment
                )
                inputs.extend(prepped_inputs)
                answers.extend(prepped_answers)

        if answer_style == "multiple_choice" or answer_style == "all":
            sentences = self._split_text(text)
            prepped_inputs, prepped_answers = self._prepare_qg_inputs_MC(
                sentences
            )
            inputs.extend(prepped_inputs)
            answers.extend(prepped_answers)

        return inputs, answers

    def generate_questions_from_inputs(self, qg_inputs: List) -> List[str]:
        """Given a list of concatenated answers and contexts, with the form:
        "answer_token <answer text> context_token <context text>", generates a list of 
        questions.
        """
        generated_questions = []

        for qg_input in qg_inputs:
            question = self._generate_question(qg_input)
            generated_questions.append(question)

        return generated_questions

    def _split_text(self, text: str) -> List[str]:
        """Splits the text into sentences, and attempts to split or truncate long sentences."""
        MAX_SENTENCE_LEN = 128
        sentences = re.findall(".*?[.!\?]", text)
        cut_sentences = []

        for sentence in sentences:
            if len(sentence) > MAX_SENTENCE_LEN:
                cut_sentences.extend(re.split("[,;:)]", sentence))

        # remove useless post-quote sentence fragments
        cut_sentences = [s for s in sentences if len(s.split(" ")) > 5]
        sentences = sentences + cut_sentences

        return list(set([s.strip(" ") for s in sentences]))

    def _split_into_segments(self, text: str) -> List[str]:
        """Splits a long text into segments short enough to be input into the transformer network.
        Segments are used as context for question generation.
        """
        MAX_TOKENS = 490
        paragraphs = text.split("\n")
        tokenized_paragraphs = [
            self.qg_tokenizer(p)["input_ids"] for p in paragraphs if len(p) > 0
        ]
        segments = []

        while len(tokenized_paragraphs) > 0:
            segment = []

            while len(segment) < MAX_TOKENS and len(tokenized_paragraphs) > 0:
                paragraph = tokenized_paragraphs.pop(0)
                segment.extend(paragraph)
            segments.append(segment)

        return [self.qg_tokenizer.decode(s, skip_special_tokens=True) for s in segments]

    def _prepare_qg_inputs(
        self,
        sentences: List[str],
        text: str
    ) -> Tuple[List[str], List[str]]:
        """Uses sentences as answers and the text as context. Returns a tuple of (model inputs, answers).
        Model inputs are "answer_token <answer text> context_token <context text>" 
        """
        inputs = []
        answers = []

        for sentence in sentences:
            qg_input = f"{self.ANSWER_TOKEN} {sentence} {self.CONTEXT_TOKEN} {text}"
            inputs.append(qg_input)
            answers.append(sentence)

        return inputs, answers

    def _prepare_qg_inputs_MC(self, sentences: List[str]) -> Tuple[List[str], List[str]]:
        """Performs NER on the text, and uses extracted entities are candidate answers for multiple-choice
        questions. Sentences are used as context, and entities as answers. Returns a tuple of (model inputs, answers). 
        Model inputs are "answer_token <answer text> context_token <context text>"
        """
        spacy_nlp = en_core_web_sm.load()
        docs = list(spacy_nlp.pipe(sentences, disable=["parser"]))
        inputs_from_text = []
        answers_from_text = []

        for doc, sentence in zip(docs, sentences):
            entities = doc.ents
            if entities:

                for entity in entities:
                    qg_input = f"{self.ANSWER_TOKEN} {entity} {self.CONTEXT_TOKEN} {sentence}"
                    answers = self._get_MC_answers(entity, docs)
                    inputs_from_text.append(qg_input)
                    answers_from_text.append(answers)

        return inputs_from_text, answers_from_text

    def _get_MC_answers(self, correct_answer: Any, docs: Any) -> List[Mapping[str, Any]]:
        """Finds a set of alternative answers for a multiple-choice question. Will attempt to find
        alternatives of the same entity type as correct_answer if possible.
        """
        entities = []

        for doc in docs:
            entities.extend([{"text": e.text, "label_": e.label_}
                            for e in doc.ents])

        # remove duplicate elements
        entities_json = [json.dumps(kv) for kv in entities]
        pool = set(entities_json)
        num_choices = (
            min(4, len(pool)) - 1
        )  # -1 because we already have the correct answer

        # add the correct answer
        final_choices = []
        correct_label = correct_answer.label_
        final_choices.append({"answer": correct_answer.text, "correct": True})
        pool.remove(
            json.dumps({"text": correct_answer.text,
                       "label_": correct_answer.label_})
        )

        # find answers with the same NER label
        matches = [e for e in pool if correct_label in e]

        # if we don't have enough then add some other random answers
        if len(matches) < num_choices:
            choices = matches
            pool = pool.difference(set(choices))
            choices.extend(random.sample(pool, num_choices - len(choices)))
        else:
            choices = random.sample(matches, num_choices)

        choices = [json.loads(s) for s in choices]

        for choice in choices:
            final_choices.append({"answer": choice["text"], "correct": False})

        random.shuffle(final_choices)
        return final_choices

    @torch.no_grad()
    def _generate_question(self, qg_input: str) -> str:
        """Takes qg_input which is the concatenated answer and context, and uses it to generate
        a question sentence. The generated question is decoded and then returned.
        """
        encoded_input = self._encode_qg_input(qg_input)
        output = self.qg_model.generate(input_ids=encoded_input["input_ids"])
        question = self.qg_tokenizer.decode(
            output[0],
            skip_special_tokens=True
        )
        return question

    def _encode_qg_input(self, qg_input: str) -> torch.tensor:
        """Tokenizes a string and returns a tensor of input ids corresponding to indices of tokens in 
        the vocab.
        """
        return self.qg_tokenizer(
            qg_input,
            padding='max_length',
            max_length=self.SEQ_LENGTH,
            truncation=True,
            return_tensors="pt",
        ).to(self.device)

    def _get_ranked_qa_pairs(
        self, generated_questions: List[str], qg_answers: List[str], scores, num_questions: int = 10
    ) -> List[Mapping[str, str]]:
        """Ranks generated questions according to scores, and returns the top num_questions examples.
        """
        if num_questions > len(scores):
            num_questions = len(scores)
            print((
                f"\nWas only able to generate {num_questions} questions.",
                "For more questions, please input a longer text.")
            )

        qa_list = []

        for i in range(num_questions):
            index = scores[i]
            qa = {
                "question": generated_questions[index].split("?")[0] + "?",
                "answer": qg_answers[index]
            }
            qa_list.append(qa)

        return qa_list

    def _get_all_qa_pairs(self, generated_questions: List[str], qg_answers: List[str]):
        """Formats question and answer pairs without ranking or filtering."""
        qa_list = []

        for question, answer in zip(generated_questions, qg_answers):
            qa = {
                "question": question.split("?")[0] + "?",
                "answer": answer
            }
            qa_list.append(qa)

        return qa_list


class QAEvaluator:
    """Wrapper for a transformer model which evaluates the quality of question-answer pairs.
    Given a QA pair, the model will generate a score. Scores can be used to rank and filter
    QA pairs.
    """

    def __init__(self) -> None:

        QAE_PRETRAINED = "iarfmoose/bert-base-cased-qa-evaluator"
        self.SEQ_LENGTH = 512

        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

        self.qae_tokenizer = AutoTokenizer.from_pretrained(QAE_PRETRAINED)
        self.qae_model = AutoModelForSequenceClassification.from_pretrained(
            QAE_PRETRAINED
        )
        self.qae_model.to(self.device)
        self.qae_model.eval()

    def encode_qa_pairs(self, questions: List[str], answers: List[str]) -> List[torch.tensor]:
        """Takes a list of questions and a list of answers and encodes them as a list of tensors."""
        encoded_pairs = []

        for question, answer in zip(questions, answers):
            encoded_qa = self._encode_qa(question, answer)
            encoded_pairs.append(encoded_qa.to(self.device))

        return encoded_pairs

    def get_scores(self, encoded_qa_pairs: List[torch.tensor]) -> List[float]:
        """Generates scores for a list of encoded QA pairs."""
        scores = {}

        for i in range(len(encoded_qa_pairs)):
            scores[i] = self._evaluate_qa(encoded_qa_pairs[i])

        return [
            k for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)
        ]

    def _encode_qa(self, question: str, answer: str) -> torch.tensor:
        """Concatenates a question and answer, and then tokenizes them. Returns a tensor of 
        input ids corresponding to indices in the vocab.
        """
        if type(answer) is list:
            for a in answer:
                if a["correct"]:
                    correct_answer = a["answer"]
        else:
            correct_answer = answer

        return self.qae_tokenizer(
            text=question,
            text_pair=correct_answer,
            padding="max_length",
            max_length=self.SEQ_LENGTH,
            truncation=True,
            return_tensors="pt",
        )

    @torch.no_grad()
    def _evaluate_qa(self, encoded_qa_pair: torch.tensor) -> float:
        """Takes an encoded QA pair and returns a score."""
        output = self.qae_model(**encoded_qa_pair)
        return output[0][0][1]


def print_qa(qa_list: List[Mapping[str, str]], show_answers: bool = True) -> None:
    """Formats and prints a list of generated questions and answers."""

    for i in range(len(qa_list)):
        # wider space for 2 digit q nums
        space = " " * int(np.where(i < 9, 3, 4))

        print(f"{i + 1}) Q: {qa_list[i]['question']}")

        answer = qa_list[i]["answer"]

        # print a list of multiple choice answers
        if type(answer) is list:

            if show_answers:
                print(
                    f"{space}A: 1. {answer[0]['answer']} "
                    f"{np.where(answer[0]['correct'], '(correct)', '')}"
                )
                for j in range(1, len(answer)):
                    print(
                        f"{space + '   '}{j + 1}. {answer[j]['answer']} "
                        f"{np.where(answer[j]['correct']==True,'(correct)', '')}"
                    )

            else:
                print(f"{space}A: 1. {answer[0]['answer']}")
                for j in range(1, len(answer)):
                    print(f"{space + '   '}{j + 1}. {answer[j]['answer']}")

            print("")

        # print full sentence answers
        else:
            if show_answers:
                print(f"{space}A: {answer}\n")


In [25]:
# Applying Model1b to check results

qg = QuestionGenerator()

context = df2['course_description'][1] # from our data I am using course_description as context

qa_list = qg.generate(
      article=context, 
      num_questions=1, # you can tweak the number of replicates here 
      answer_style='sentences' # could be sentences, multiple_choice or all (for mixed)
      )

print('The generated question and context is: ')
print_qa(qa_list)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Generating questions...

Evaluating QA pairs...

The generated question and context is: 
1) Q: What is the core language for Big Data analysis?
   A: Master SQL, the core language for Big Data analysis, and enable insight-driven decision-making and strategy for your business.



# Finally I chose to generate questions using Model 1a


In [26]:
# Both models seem good to generate a reasonable question - I will use Model1a 
# If "num_questions" above is more than 1, below code should change to account for this

generated_questions = [] # empty list to save the results and append to the dataframe

# loop over the dataframe to generate a question for each course_description / answer
for i, val in enumerate(df2['course_description']):
  context = val
  answer = df2['course_name'][i]
  generated_questions.append(get_question(answer, context)) # saving the generated questions in this list

df2['generated_questions'] = generated_questions # adding the column to the dataframe

/var/folders/7h/l_zr79r1053dk59rg6fl_cnc0000gn/T/ipykernel_42904/3360590561.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['generated_questions'] = generated_questions # adding the column to the dataframe


In [27]:
df2.head()

,learning_platform,career_path,skill_level,course_name,course_description,generated_questions
0,Udacity,Data Science,Beginner,Intro to Data Science,The Introduction to Data Science class will su...,<pad> question: What class will focus on the f...
1,Udacity,Data Science,Beginner,Learn SQL,"Master SQL, the core language for Big Data ana...",<pad> question: What is the core language for ...
2,Udacity,Data Science,Beginner,Business Analytics Online Course,Gain foundational data skills applicable to an...,<pad> question: What is the name of the course...
3,Udacity,Data Science,Beginner,Learn Python for Data Science,Prepare for a data science career by learning ...,<pad> question: What is the basic data program...
4,Coursera,Data Science,Beginner,Python for Everybody,This Specialization builds on the success of t...,<pad> question: What course did the Capstone P...


In [28]:
import re 

In [29]:
df3 = df2.copy() # Backup

# removing unwanted characters
for i, val in enumerate(df3['generated_questions']): 
  df3['generated_questions'][i] = re.sub(r'^.*?W', 'W', val)
  df3['generated_questions'][i] = df3['generated_questions'][i].split("<")[0]

In [30]:
df3['generated_questions'][0] # check

'What class will focus on the foundational topics in data science?'

# **Using FAISS**
To link user question to most similar generated question and then provide course details from our dataset



In [ ]:
%%capture
!pip install sentence-transformers
!pip install faiss-gpu # or faiss-cpu could be used

In [ ]:
# Here I am mainly changing the generated question text into a numerical vector to be used for similarity

%%capture

import faiss
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

data = df3 # our main data

model = SentenceTransformer('stsb-mpnet-base-v2') # to work with similarity sentences needed to be changed to numerical vectors
                                                  # I chose this model to encode the sentences

encoded_data = model.encode(data['generated_questions']) # I use the above sentence transformer to encode the data into a numerical vector

In [ ]:
# FAISS is similar to a search engine, so here I am building an index using the above question vectors
# when the user asks a question I search within the index for the most similar question and give a detailed answer

dim = encoded_data.shape[1] # shape is used to get the vectors' dimension

embeddings = np.array([embedding for embedding in encoded_data]).astype("float32") # changing data type to float 32

index = faiss.IndexIDMap(faiss.IndexFlatL2(dim)) # building the index 

index.add_with_ids(embeddings, np.array(range(0, len(data['generated_questions'])))) # mapping ids to the indexed vectors

#faiss.write_index(index, os.path.join(index_path, index_name+'.index')) # You can also save the index by writing the index to the file path and calling it later

# Example 1

In [ ]:
# Now when the user inputs a question I could look for the most similar question(s) and give an answer

query = 'What is a good big data course?' # You can try any other query
k = 2 # could change to find the most similar k number of questions

query_vector = model.encode([query]) # The user query also needs to be changed to a numerical vector for comparison

top_k = index.search(query_vector, k)

In [ ]:
# now we can get the most similar question from the dataframe we had
import pprint

loc_df = top_k[1][0] # to extract the location in our dataframe

for i in loc_df: # for loop if above k is larger than one
  print('The user asked: ', query)
  print('The most similar question is: ', data['generated_questions'][i])
  print('The course name is: ', data['course_name'][i])
  print('The course description is: ')
  pprint.pprint(data['course_description'][i])
  print('The career path is: ', data['career_path'][i])
  print('The learning platform is: ', data['learning_platform'][i])
  print('The skill level is: ', data['skill_level'][i])
  print('-'*50)

The user asked:  What is a good big data course?
The most similar question is:  What is the name of the course that you can take to learn how to use data?
The course name is:  Business Analytics Online Course 
The course description is: 
('Gain foundational data skills applicable to any industry. Collect and '
 'analyze data, model business scenarios, and communicate your findings with '
 'SQL, Excel, and Tableau.')
The career path is:  Data Science
The learning platform is:  Udacity
The skill level is:  Beginner
--------------------------------------------------
The user asked:  What is a good big data course?
The most similar question is:  What is the name of the course that will help you to become a data scientist?
The course name is:  How to Become a Data Scientist 
The course description is: 
('Gain real-world data science experience with projects designed by industry '
 'experts. Build your portfolio and advance your data science career.')
The career path is:  Data Science
The le

# Example 2

In [ ]:
# Now when the user inputs a question I could look for the most similar question(s) and give an answer

query = 'I want to learn about SQL' # You can try any other query
k = 2 # could change to find the most similar k number of questions

query_vector = model.encode([query]) # The user query also needs to be changed to a numerical vector for comparison

top_k = index.search(query_vector, k)

In [ ]:
# now we can get the most similar question from the dataframe we had

loc_df = top_k[1][0] # to extract the location in our dataframe

for i in loc_df: # for loop if above k is larger than one
  print('The user asked: ', query)
  print('The most similar question is: ', data['generated_questions'][i])
  print('The course name is: ', data['course_name'][i])
  print('The course description is: ')
  pprint.pprint(data['course_description'][i])
  print('The career path is: ', data['career_path'][i])
  print('The learning platform is: ', data['learning_platform'][i])
  print('The skill level is: ', data['skill_level'][i])
  print('-'*50)

The user asked:  I want to learn about SQL
The most similar question is:  What is the name of the course that will help you to become a data scientist?
The course name is:  How to Become a Data Scientist 
The course description is: 
('Gain real-world data science experience with projects designed by industry '
 'experts. Build your portfolio and advance your data science career.')
The career path is:  Data Science
The learning platform is:  Udacity
The skill level is:  Intermediate
--------------------------------------------------
The user asked:  I want to learn about SQL
The most similar question is:  What is the term for the skills needed to leverage the power of Data Science in your business?
The course name is:  Data Science for Business & Business Leaders
The course description is: 
('Master the strategic decision-making skills for the people, platforms, and '
 'processes required to leverage the power of Data Science in your business.')
The career path is:  Data Science
The lea

# **Another FAISS strategy** -- I noticed from Example 2 that there are better SQL courses, maybe the *course description* could be a good similarity measure

# You could see from the two strategies that there are advantage and drawbacks to each one. 

# Note, the bigger our data, the more accurate the results should be

# New Strategy Ex. 1

In [ ]:
encoded_data2 = model.encode(data['course_description']) # I use the above sentence transformer to encode the data into a numerical vector

In [ ]:
# FAISS is similar to a search engine, so here I am building an index using the above question vectors
# when the user asks a question I search within the index for the most similar question and give a detailed answer

dim2 = encoded_data2.shape[1] # shape is used to get the vectors' dimension

embeddings2 = np.array([embedding for embedding in encoded_data2]).astype("float32") # changing data type to float 32

index2 = faiss.IndexIDMap(faiss.IndexFlatL2(dim)) # building the index 

index2.add_with_ids(embeddings2, np.array(range(0, len(data['course_description'])))) # mapping ids to the indexed vectors

In [ ]:
# Now when the user inputs a question I could look for the most similar question(s) and give an answer

query_new = 'What is a good big data course?' # You can try any other query
k_new = 2 # could change to find the most similar k number of questions

query_vector_new = model.encode([query]) # The user query also needs to be changed to a numerical vector for comparison

top_k_new = index2.search(query_vector, k)

In [ ]:
# now we can get the most similar question from the dataframe we had

loc_df2 = top_k_new[1][0] # to extract the location in our dataframe

for i in loc_df2: # for loop if above k is larger than one
  print('The user asked: ', query_new)
  print('The course name is: ', data['course_name'][i])
  print('The course description is: ')
  pprint.pprint(data['course_description'][i])
  print('The career path is: ', data['career_path'][i])
  print('The learning platform is: ', data['learning_platform'][i])
  print('The skill level is: ', data['skill_level'][i])
  print('-'*50)

The user asked:  What is a good big data course?
The course name is:  Learn SQL 
The course description is: 
('Master SQL, the core language for Big Data analysis, and enable '
 'insight-driven decision-making and strategy for your business.')
The career path is:  Data Science
The learning platform is:  Udacity
The skill level is:  Beginner
--------------------------------------------------
The user asked:  What is a good big data course?
The course name is:  Learn R Programming 
The course description is: 
('Prepare for a data science career by learning the fundamental data '
 'programming tools: R, SQL, command line, and git.')
The career path is:  Data Science
The learning platform is:  Udacity
The skill level is:  Intermediate
--------------------------------------------------


# New Strategy Ex.2

In [ ]:
# Now when the user inputs a question I could look for the most similar question(s) and give an answer

query_new = 'I want to learn about SQL' # You can try any other query
k_new = 2 # could change to find the most similar k number of questions

query_vector_new = model.encode([query]) # The user query also needs to be changed to a numerical vector for comparison

top_k_new = index2.search(query_vector, k)

In [ ]:
# now we can get the most similar question from the dataframe we had

loc_df2 = top_k_new[1][0] # to extract the location in our dataframe

for i in loc_df2: # for loop if above k is larger than one
  print('The user asked: ', query_new)
  print('The course name is: ', data['course_name'][i])
  print('The course description is: ')
  pprint.pprint(data['course_description'][i])
  print('The career path is: ', data['career_path'][i])
  print('The learning platform is: ', data['learning_platform'][i])
  print('The skill level is: ', data['skill_level'][i])
  print('-'*50)

The user asked:  I want to learn about SQL
The course name is:  Learn SQL 
The course description is: 
('Master SQL, the core language for Big Data analysis, and enable '
 'insight-driven decision-making and strategy for your business.')
The career path is:  Data Science
The learning platform is:  Udacity
The skill level is:  Beginner
--------------------------------------------------
The user asked:  I want to learn about SQL
The course name is:  Learn R Programming 
The course description is: 
('Prepare for a data science career by learning the fundamental data '
 'programming tools: R, SQL, command line, and git.')
The career path is:  Data Science
The learning platform is:  Udacity
The skill level is:  Intermediate
--------------------------------------------------
